<a href="https://colab.research.google.com/github/bobbyrayyy/mozillaTTS/blob/main/TTS_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Hands-on example for TTS  [https://github.com/mozilla/TTS](https://github.com/mozilla/TTS)

This notebook trains Tacotron model on LJSpeech dataset.

In [1]:
# download LJSpeech dataset
!wget http://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
# decompress
!tar -xjf LJSpeech-1.1.tar.bz2

--2021-01-08 07:46:25--  http://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
Resolving data.keithito.com (data.keithito.com)... 174.138.79.61
Connecting to data.keithito.com (data.keithito.com)|174.138.79.61|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2748572632 (2.6G) [application/octet-stream]
Saving to: ‘LJSpeech-1.1.tar.bz2’

LJSpeech-1.1.tar.bz 100%[===================>]   2.56G  48.8MB/s    in 54s     

2021-01-08 07:47:19 (48.2 MB/s) - ‘LJSpeech-1.1.tar.bz2’ saved [2748572632/2748572632]



In [2]:
# create train-val splits
!shuf LJSpeech-1.1/metadata.csv > LJSpeech-1.1/metadata_shuf.csv
!head -n 12000 LJSpeech-1.1/metadata_shuf.csv > LJSpeech-1.1/metadata_train.csv
!tail -n 1100 LJSpeech-1.1/metadata_shuf.csv > LJSpeech-1.1/metadata_val.csv

In [17]:

%cd
%cd /content/TTS/TTS/bin
!ls
!pip install -r /content/TTS/requirements.txt


/root
/content/TTS/TTS/bin
compute_embeddings.py		  distribute.py      train_vocoder_gan.py
compute_statistics.py		  distribute_tts.py  train_vocoder_wavegrad.py
config.json			  scale_stats.npy    train_vocoder_wavernn.py
convert_melgan_tflite.py	  synthesize.py      tts_model.pth.tar
convert_melgan_torch_to_tf.py	  train_encoder.py   tune_wavegrad.py
convert_tacotron2_tflite.py	  train_glow_tts.py
convert_tacotron2_torch_to_tf.py  train_tacotron.py


In [18]:
# get TTS to your local
!ls
!git clone https://github.com/mozilla/TTS

compute_embeddings.py		  distribute.py      train_vocoder_gan.py
compute_statistics.py		  distribute_tts.py  train_vocoder_wavegrad.py
config.json			  scale_stats.npy    train_vocoder_wavernn.py
convert_melgan_tflite.py	  synthesize.py      tts_model.pth.tar
convert_melgan_torch_to_tf.py	  train_encoder.py   tune_wavegrad.py
convert_tacotron2_tflite.py	  train_glow_tts.py
convert_tacotron2_torch_to_tf.py  train_tacotron.py
Cloning into 'TTS'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 10320 (delta 9), reused 12 (delta 4), pack-reused 10300
Receiving objects: 100% (10320/10320), 120.37 MiB | 32.01 MiB/s, done.
Resolving deltas: 100% (7185/7185), done.


In [20]:
# install espeak backend if you like to use phonemes instead of raw characters
!sudo apt-get install espeak
!pip install soundfile

Reading package lists... Done
Building dependency tree       
Reading state information... Done
espeak is already the newest version (1.48.04+dfsg-5).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.


In [21]:
!cd
!ls


compute_embeddings.py		  distribute.py      train_vocoder_gan.py
compute_statistics.py		  distribute_tts.py  train_vocoder_wavegrad.py
config.json			  scale_stats.npy    train_vocoder_wavernn.py
convert_melgan_tflite.py	  synthesize.py      TTS
convert_melgan_torch_to_tf.py	  train_encoder.py   tts_model.pth.tar
convert_tacotron2_tflite.py	  train_glow_tts.py  tune_wavegrad.py
convert_tacotron2_torch_to_tf.py  train_tacotron.py


In [22]:
%cd TTS

/content/TTS/TTS/bin/TTS


In [23]:
# install TTS requirements
!python setup.py install

Compiling ./TTS/tts/layers/glow_tts/monotonic_align/core.pyx because it changed.
[1/1] Cythonizing ./TTS/tts/layers/glow_tts/monotonic_align/core.pyx
running install
running bdist_egg
running egg_info
creating TTS.egg-info
writing TTS.egg-info/PKG-INFO
writing dependency_links to TTS.egg-info/dependency_links.txt
writing entry points to TTS.egg-info/entry_points.txt
writing requirements to TTS.egg-info/requires.txt
writing top-level names to TTS.egg-info/top_level.txt
writing manifest file 'TTS.egg-info/SOURCES.txt'
writing manifest file 'TTS.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
-- Building version 0.0.6+9cf474a
creating temp_build
creating temp_build/TTS
copying TTS/__init__.py -> temp_build/TTS
creating temp_build/TTS/tts
copying TTS/tts/__init__.py -> temp_build/TTS/tts
creating temp_build/TTS/server
copying TTS/server/synthesizer.py -> temp_build/TTS/server
copying TTS/server/__init__.py -> temp_build/TTS

In [24]:

!ls


import json
from TTS.utils.io import load_config
CONFIG = load_config('/content/TTS/TTS/tts/configs/config.json')
CONFIG['datasets'][0]['path'] = '../LJSpeech-1.1/'
CONFIG['output_path'] = '../'
with open('/content/TTS/TTS/tts/configs/config.json', 'w') as fp:
    json.dump(CONFIG, fp)


build		    Dockerfile	 requirements_tests.txt  temp_build
CODE_OF_CONDUCT.md  images	 requirements.txt	 tests
CODE_OWNERS.rst     LICENSE.txt  run_tests.sh		 TTS
CONTRIBUTING.md     notebooks	 setup.cfg		 TTS.egg-info
dist		    README.md	 setup.py		 version.py


In [25]:
# Download model, config.json, and vocoder
!gdown --id 1GMZlB9kwXbji-TSOsxGlPDDUZ7V-k-JP -O tts_model.pth.tar
!gdown --id 1CpnesSgHBUFxK3tiUl_ExprnL8LAE0lc -O config.json
!gdown --id 1D-ImP_0REVKJ9qlco5JOwxgTCa9L0Uiq -O scale_stats.npy

Downloading...
From: https://drive.google.com/uc?id=1GMZlB9kwXbji-TSOsxGlPDDUZ7V-k-JP
To: /content/TTS/TTS/bin/TTS/tts_model.pth.tar
302MB [00:02, 114MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1CpnesSgHBUFxK3tiUl_ExprnL8LAE0lc
To: /content/TTS/TTS/bin/TTS/config.json
100% 7.66k/7.66k [00:00<00:00, 12.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1D-ImP_0REVKJ9qlco5JOwxgTCa9L0Uiq
To: /content/TTS/TTS/bin/TTS/scale_stats.npy
100% 10.5k/10.5k [00:00<00:00, 17.0MB/s]


In [29]:
# pull the trigger
# !python train_glow_tts.py --config_path /content/config.json
!python /content/TTS/TTS/bin/train_glow_tts.py --config_path /content/TTS/TTS/bin/config.json --restore_path /content/TTS/TTS/bin/tts_model.pth.tar
# !python /content/TTS/TTS/bin/train_tacotron.py --config_path /content/TTS/TTS/tts/configs/config.json --restore_path /content/TTS/TTS/bin/tts_model.pth.tar

2021-01-08 08:48:49.032640: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
 > Using CUDA:  True
 > Number of GPUs:  1
   > Mixed precision enabled.
 > Git Hash: 9cf474a
 > Experiment folder: /content/mozilla/output/glow-tts-gatedconv-January-08-2021_08+48AM-9cf474a
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > num_mels:80
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:0
 | > fft_size:1024
 | > power:1.1
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:50.0
 | > mel_fmax:7600.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > stats_path:/content/TTS/TTS/bin/scale_stats.npy
 | > hop_length:256
 | > win_length:1024
 | > Found 13100 files in /content/LJSpeech-1.1
 > Using model: glow_tts
 ! Run is 

In [15]:
!pip list

Package                       Version        
----------------------------- ---------------
absl-py                       0.10.0         
alabaster                     0.7.12         
albumentations                0.1.12         
altair                        4.1.0          
argon2-cffi                   20.1.0         
asgiref                       3.3.1          
astor                         0.8.1          
astroid                       2.4.2          
astropy                       4.1            
astunparse                    1.6.3          
async-generator               1.10           
atari-py                      0.2.6          
atomicwrites                  1.4.0          
attrdict                      2.0.1          
attrs                         20.3.0         
audioread                     2.1.9          
autograd                      1.3            
Babel                         2.9.0          
backcall                      0.2.0          
beautifulsoup4                4.6.